In [6]:
import h5py
import torch
from src.data.play import Play, slice_keys
from src.utilities.embedding_utilities import n_players
from torch.nn import Embedding
from torch import Tensor as t

In [7]:
db = h5py.File("cache/full_db.h5", "r")

In [8]:
i = 0
plays = []
for p in db["2016_regular/plays"][0:10]:
    play = Play(p)
    plays.append(play)

In [14]:
data = play.data

new_play = Play(data)
play.shot_distance

tensor(40.)

In [ ]:
play

In [5]:
print(play.triggers)

{'shooter': [{'initial_event': 'shot'}], 'shot_made': [{'initial_event': 'shot'}], 'is_3': [{'initial_event': 'shot'}], 'shot_distance': [{'initial_event': 'shot'}], 'shot_fouled': [{'initial_event': 'shot'}], 'shooting_fouler': [{'shot_fouled': True}], 'assisted': [{'initial_event': 'shot'}], 'assister': [{'initial_event': 'shot'}], 'blocked': [{'shot_made': False}], 'blocker': [{'blocked': True}], 'turnoverer': [{'initial_event': 'turnover'}], 'offensive_foul': [{'initial_event': 'turnover'}], 'offensive_fouler': [{'offensive_foul': True}], 'offensive_foul_drawer': [{'offensive_foul': True}], 'stolen': [{'initial_event': 'turnover'}], 'stealer': [{'stolen': True}], 'over_limit_fouler': [{'initial_event': 'foul_over_limit'}], 'over_limit_foul_drawer': [{'initial_event': 'foul_over_limit'}], 'first_free_throw_made': [{'initial_event': 'foul_over_limit'}, {'shot_fouled': True, 'shot_made': False}], 'middle_free_throw_made': [{'shot_fouled': True, '3_point_shot': True, 'shot_made': False

In [15]:
from collections import defaultdict
items_per_type = defaultdict(list)
for k, v in play.types.items():
    items_per_type[v].append(k)
items_per_type = dict(items_per_type)
print(items_per_type)

{'embedding': ['offense_team', 'defense_team', 'shooter', 'shooting_fouler', 'assister', 'blocker', 'turnoverer', 'offensive_fouler', 'offensive_foul_drawer', 'stealer', 'common_fouler', 'common_foul_drawer', 'over_limit_fouler', 'over_limit_foul_drawer', 'free_thrower', 'offensive_rebounder', 'defensive_rebounder'], 'embedding_list': ['offense_roster', 'defense_roster'], 'bool': ['is_second_chance', 'shot_made', 'is_3', 'shot_fouled', 'assisted', 'blocked', 'offensive_foul', 'stolen', 'first_free_throw_made', 'middle_free_throw_made', 'last_free_throw_made'], 'choice': ['initial_event', 'shot_type', 'rebound_type'], 'scalar': ['shot_distance']}


In [18]:
bools = t(len(items_per_type["bool"])).float()
print(bools)


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [39]:
import oyaml
class DataConfig():
    def __init__(self, yaml_path):
        path = oyaml.load(open(yaml_path, "r"), Loader=oyaml.Loader)

        self.data_keys = []
        self.triggers = {}
        self.types = {}
        self.is_embedding = {}
        self.is_int = {}
        self.slice_keys = {}
        self.data_indices = {}
        self.choice_keys = {}
        self.choice_indices = {}
        for k, v in path.items():
            if v['type'] == 'embedding_list':
                self.is_embedding[k] = True
                self.is_int[k] = False
                for i in range(v['len']):
                    sub_key = k + "." + str(i)
                    self.data_keys.append(sub_key)
                    self.data_indices[sub_key] = len(self.data_keys)
                    self.is_embedding[sub_key] = True
                    self.is_int[sub_key] = False
                start = self.data_indices[k + ".0"]
                end = len(self.data_keys)
                self.slice_keys[k] = slice(start, end+1)
            elif v['type'] == 'choice':
                self.is_embedding[k] = False
                self.is_int[k] = False
                if type(v['choices']) is list:
                    choices = v['choices']
                else:
                    choice_range = list(range(
                        self.slice_keys[v['choices']].start,
                        self.slice_keys[v['choices']].stop + 1
                    ))
                    choices = [self.data_keys[i] for i in choice_range]
                for choice in choices:
                    sub_key = k + "." + str(choice)
                    self.data_keys.append(sub_key)
                    self.data_indices[sub_key] = len(self.data_keys)
                    self.is_embedding[sub_key] = False
                    self.is_int[sub_key] = False
                start = self.data_indices[k + "." + str(choices[0])] 
                end = len(self.data_keys)
                self.slice_keys[k] = slice(start, end+1)
                self.choice_keys[k] = choices
                self.choice_indices[k] = {choice: i for i, choice in enumerate(choices)}
            else:
                self.data_keys.append(k)
                self.is_embedding[k] = (v['type'] == 'embedding')
                self.is_int[k] = (v['type'] == 'int')

            if 'triggers' in v:
                self.triggers[k] = v['triggers']
            self.types[k] = v['type']

In [37]:
d = DataConfig("src/data/play.yaml")
d.data_keys
d.slice_keys

{'offense_team': 'embedding',
 'defense_team': 'embedding',
 'offense_roster': 'embedding_list',
 'defense_roster': 'embedding_list',
 'is_second_chance': 'bool',
 'initial_event': 'choice',
 'shooter': 'embedding',
 'shot_made': 'bool',
 'is_3': 'bool',
 'shot_distance': 'scalar',
 'shot_type': 'choice',
 'shot_fouled': 'bool',
 'shooting_fouler': 'embedding',
 'assisted': 'bool',
 'assister': 'embedding',
 'blocked': 'bool',
 'blocker': 'embedding',
 'turnoverer': 'embedding',
 'offensive_foul': 'bool',
 'offensive_fouler': 'embedding',
 'offensive_foul_drawer': 'embedding',
 'stolen': 'bool',
 'stealer': 'embedding',
 'common_fouler': 'embedding',
 'common_foul_drawer': 'embedding',
 'over_limit_fouler': 'embedding',
 'over_limit_foul_drawer': 'embedding',
 'first_free_throw_made': 'bool',
 'middle_free_throw_made': 'bool',
 'last_free_throw_made': 'bool',
 'free_thrower': 'embedding',
 'rebound_type': 'choice',
 'offensive_rebounder': 'embedding',
 'defensive_rebounder': 'embedding